In [49]:
from abi import Abi
import pandas as pd
import requests
import json
from _const import * # paths 
from _func import * # lambdas
pd.set_option('display.max_rows', 500)


In [50]:
queries_list = PATHS.QUERIES.ls()
queries_list

[WindowsPath('queries/guardians_count.graphql')]

In [51]:
queries_dict = {}

for q in queries_list:
    q_name = q.parts[-1].split(".")[0]
    with open(q) as f:
        q_txt = f.read()


    queries_dict[q_name] = q_txt

In [52]:
queries_dict['guardians_count']

'{\n  guardians {\n    id\n    treeId # registry.getJurorId(jurorAddress) \n    activeBalance\n    lockedBalance\n    availableBalance\n    deactivationBalance\n    withdrawalsLockTermId\n    disputes\n    drafts\n    stakingMovements\n    shareClaims\n    createdAt # event.block.timestamp\n    }\n  \n}'

In [53]:

query = """{
  courts(first: 5) {
    id
    token {
      id
    }
    currentTerm
    termDuration
  }
  courtModules(first: 5) {
    id
    court {
      id
    }
    moduleId
    type
  }
}"""

In [54]:
import json
r = requests.post(
    URLS.SUBGRAPH.COURT.MAINNET,
    json={'query': queries_dict['guardians_count']})

json_data = json.loads(r.text)

In [55]:
json_data['data'].keys()

dict_keys(['guardians'])

In [56]:
df_guardians = pd.DataFrame(json_data['data']['guardians'])
df_guardians.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 96 entries, 0 to 95
Data columns (total 12 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   activeBalance          96 non-null     object
 1   availableBalance       96 non-null     object
 2   createdAt              96 non-null     object
 3   deactivationBalance    96 non-null     object
 4   disputes               96 non-null     object
 5   drafts                 96 non-null     object
 6   id                     96 non-null     object
 7   lockedBalance          96 non-null     object
 8   shareClaims            96 non-null     object
 9   stakingMovements       96 non-null     object
 10  treeId                 96 non-null     object
 11  withdrawalsLockTermId  96 non-null     object
dtypes: object(12)
memory usage: 9.1+ KB


In [57]:
df_guardians.head(100)

,activeBalance,availableBalance,createdAt,deactivationBalance,disputes,drafts,id,lockedBalance,shareClaims,stakingMovements,treeId,withdrawalsLockTermId
0,880828164821533660463,0,1628932243,0,[],[],0x01704865b3404391ea00137c4bb00855515df802,0,[],"[{}, {}]",83,0
1,811700000000000000000,0,1628229988,0,[],[],0x03000a0ecb1898040c26cd418570cd480775f0da,0,[],"[{}, {}, {}, {}]",34,0
2,33875561690053511743407,0,1628945946,0,[],[],0x06b172c63730f0ccfb40ea35e726ebf4e930c9e3,0,[],"[{}, {}]",85,0
3,135421986952300156491,0,1628186705,0,[],[],0x098d9a770f3bd88bef60e72c833ab8db240d5928,0,[],"[{}, {}]",25,0
4,4846740038258336017978,0,1628207854,0,[],[],0x0a3bfd31570df77c243379606ef1fca7ddd2ec74,0,[],"[{}, {}]",29,0
5,249297594415196945423,0,1628213740,0,[],[],0x104e4df284dee7547d3f18e8f6553561abf1fa29,0,[],"[{}, {}]",31,0
6,8406823277560969281702,0,1628742243,0,[],[],0x10b4aef445821f70ca48bf0a5ef1a06eb815eb21,0,[],"[{}, {}]",76,0
7,1410836577815865841719,0,1628382529,0,[],[],0x145044e79da7d462224a1af5121543ef6ef2508a,0,[],"[{}, {}]",50,0
8,1848241878572513272322,0,1628189500,0,[],[],0x152e59fc713ddc6885eaee29701da6552807435c,0,[],"[{}, {}]",26,0
9,110000000000000000000,0,1628090251,0,[],[],0x1555d9dd364168ce671a92493aa01f05caf68cb4,0,[],"[{}, {}]",3,0


In [48]:
for col in df_guardians.columns:
    print(col)
    print(df_guardians[col].value_counts().head(5))
    print()

activeBalance
110000000000000000000     2
440000000000000000000     2
4400000000000000000000    2
880828164821533660463     1
4830408847925143582427    1
Name: activeBalance, dtype: int64

availableBalance
0    96
Name: availableBalance, dtype: int64

createdAt
1628932243    1
1628229988    1
1628504782    1
1628588227    1
1628336355    1
Name: createdAt, dtype: int64

deactivationBalance
0    96
Name: deactivationBalance, dtype: int64

disputes
[]      93
[{}]     3
Name: disputes, dtype: int64

drafts
[]      93
[{}]     3
Name: drafts, dtype: int64

id
0x01704865b3404391ea00137c4bb00855515df802    1
0x03000a0ecb1898040c26cd418570cd480775f0da    1
0xbbb0592561d803255760e627a55a1458450613a0    1
0xb62e27446079c2f2575c79274cd905bf1e1e4edb    1
0xb2ceb04874fce4cefd38c751c8ed7b88d2185c4d    1
Name: id, dtype: int64

lockedBalance
0                       93
30000000000000000000     3
Name: lockedBalance, dtype: int64

shareClaims
[]    96
Name: shareClaims, dtype: int64

stakingMovements